In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("input/heart.csv")

In [11]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


# Attributes
1. Age
2. Sex
3. Chest pain type (cp)
  * 4 types
4. Resting blood presure (trestbps)
5. Serum cholestoral in mg/dl (chol)
6. Fasting blood sugar > 120 mg/dl (fbs)
7. Resting electrocardiographic results (restecg)
  * Values 0-2
8. Maximum heart rate achieved (thalach)
9. Exercise induced angina (exang)
  * 1 = yes; 0 = no
10. ST depression induced by exercise relative to rest (oldpeak)
11. Slope of the peak exercise ST segment (slope)
12. Number of major vessels colored by flourosopy
  * 0-3
13. thal
  * 3 = normal
  * 6 = fixed defect
  * 7 = reversable defect

In [5]:
df.sample(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
51,66,1,0,120,302,0,0,151,0,0.4,1,0,2,1
213,61,0,0,145,307,0,0,146,1,1.0,1,0,3,0
162,41,1,1,120,157,0,1,182,0,0.0,2,0,2,1


# Looking at Categorical Attributes:
### "cp", "fbs", "restecg", "ca", "thal"

In [100]:
# cp_cat = (df.groupby(['target']))['cp'].value_counts(normalize=True).mul(100)
df.groupby(['target'])['cp'].value_counts(normalize=True).mul(100)

target  cp
0       0     75.362319
        2     13.043478
        1      6.521739
        3      5.072464
1       2     41.818182
        1     24.848485
        0     23.636364
        3      9.696970
Name: cp, dtype: float64

In [34]:
cp_cat[:]

,target,cp,percentage
0,0,0,75.362319
1,0,2,13.043478
2,0,1,6.521739
3,0,3,5.072464
4,1,2,41.818182
5,1,1,24.848485
6,1,0,23.636364
7,1,3,9.696970
